In [2]:
import numpy as np
import pandas as pd

In [3]:
df = pd.read_table('MovieSummaries/character.metadata.tsv')
print(df)

          975900   /m/03vyhn  2001-08-24                    Akooshay  \
0         975900   /m/03vyhn  2001-08-24  Lieutenant Melanie Ballard   
1         975900   /m/03vyhn  2001-08-24         Desolation Williams   
2         975900   /m/03vyhn  2001-08-24          Sgt Jericho Butler   
3         975900   /m/03vyhn  2001-08-24             Bashira Kincaid   
4         975900   /m/03vyhn  2001-08-24   Commander Helena Braddock   
...          ...         ...         ...                         ...   
450663    913762   /m/03pcrp  1992-05-21                      Elensh   
450664    913762   /m/03pcrp  1992-05-21                      Hibiki   
450665  28308153  /m/0cp05t9        1957                         NaN   
450666  28308153  /m/0cp05t9        1957                         NaN   
450667  28308153  /m/0cp05t9        1957                         NaN   

        1958-08-26    F   1.62  Unnamed: 7      Wanda De Jesus    42  \
0       1974-08-15    F  1.780  /m/044038p  Natasha Henstridge 